# Database preparation

## create database

In [102]:
import sqlite3
from datetime import datetime
import json
YOUTUBE_DBNAME = 'youtube.sqlite'
TOP_VIDEOS = "top_videos.json"

In [117]:
# create database for youtube
conn = sqlite3.connect(YOUTUBE_DBNAME)
cur = conn.cursor()

# channels table
drop_channels = '''
  DROP TABLE IF EXISTS "channels";
'''

create_channels = '''
  CREATE TABLE IF NOT EXISTS "channels" (
    channelId TEXT PRIMARY KEY,
    title TEXT NOT NULL,
    country TEXT,
    createTime TEXT,
    subscriberCount TEXT,
    videoCount TEXT,
    viewCount TEXT
  );
'''

# playlists table
drop_playlists = '''
  DROP TABLE IF EXISTS "playlists";
'''

create_playlists = '''
  CREATE TABLE IF NOT EXISTS "playlists" (
    playlistId TEXT PRIMARY KEY,
    title TEXT NOT NULL,
    createTime TEXT,
    videoCount Integer,
    channelId TEXT,
    FOREIGN KEY(channelId) REFERENCES channels(channelId)
  )
'''

# videos table
drop_videos = '''
  DROP TABLE IF EXISTS "videos";
'''

create_videos = '''
  CREATE TABLE IF NOT EXISTS "videos" (
    videoId TEXT PRIMARY KEY,
    title TEXT NOT NULL,
    categoryId INTEGER,
    viewCount TEXT,
    likeCount TEXT,
    dislikeCount TEXT,
    favoriteCount TEXT,
    commentCount TEXT,
    playlistId TEXT,
    FOREIGN KEY(playlistId) REFERENCES playlists(playlistId)
  )
'''

# create
conn.execute(drop_channels)
conn.execute(create_channels)
conn.execute(drop_playlists)
conn.execute(create_playlists)
conn.execute(drop_videos)
conn.execute(create_videos)
conn.close()

## create database manipulation functions

In [118]:
# insert a "video list" return result into videos table
def insert_to_videos(video):
  conn = sqlite3.connect(YOUTUBE_DBNAME)
  cur = conn.cursor()
  cmd = 'INSERT INTO videos VALUES(?,?,?,?,?,?,?,?)'
  video_info = [video['id'], video['snippet']['title'], video['snippet']['categoryId'],video['statistics']['viewCount'], video['statistics']['likeCount'], 
                video['statistics']['dislikeCount'],video['statistics']['favoriteCount'],video['statistics'].get('commentCount', 'UNKNOWN'), video['snippet']['playlistId']]
  cur.execute(cmd, video_info)
  conn.commit()
  conn.close()

In [123]:
# insert a "channel list" return result into channels table
def insert_to_channels(channel):
  conn = sqlite3.connect(YOUTUBE_DBNAME)
  cur = conn.cursor()
  cmd = 'INSERT INTO channels VALUES(?,?,?,?,?,?,?)'
  channel_info = [channel['id'], channel['snippet']['title'], channel['snippet']['country'], channel['snippet']['publishedAt'], 
                  channel['statistics']['subscriberCount'], channel['statistics']['videoCount'], channel['statistics']['viewCount']]
  cur.execute(cmd, channel_info)
  conn.commit()
  conn.close()

In [124]:
# insert a "playlist list" return result into playlistss table
def insert_to_playlists(playlist):
  conn = sqlite3.connect(YOUTUBE_DBNAME)
  cur = conn.cursor()
  cmd = 'INSERT INTO playlists VALUES(?,?,?,?,?,?,?)'
  playlist_info = [playlist['id'], playlist['snippet']['title'], playlist['snippet']['publishedAt'], playlist['contentDetails']['itemCount'],
                   playlist['snippet']['channelId']]
  cur.execute(cmd, playlist_info)
  conn.commit()
  conn.close()

# Twitter API

In [1]:
import requests
import tweepy
import twitter_secrets

In [2]:
auth = tweepy.OAuthHandler(twitter_secrets.TWITTER_API_KEY, twitter_secrets.TWITTER_API_SECRET)
auth.set_access_token(twitter_secrets.TWITTER_ACCESS_TOKEN, twitter_secrets.TWITTER_ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

In [3]:
rdt = api.get_user("realDonaldTrump")

In [6]:
rdt_timeline = rdt.timeline()

In [13]:
print(rdt_timeline[0].text)

See you tomorrow night at 7PM, Valdosta, Georgia! https://t.co/Or9WioLiVk


# Youtube API

## preparation

In [14]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

import json

In [15]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "client_secret_1089772597554-c2p8be1ca47tbg8cmedstjr18iqcac4m.apps.googleusercontent.com.json"

# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1089772597554-c2p8be1ca47tbg8cmedstjr18iqcac4m.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=vpOlQb1beLojQbR4GVFbxb7WOVU8FM&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g6TZj_0BaFWSPq202t88ARZemfKn6IZQ7gmgpAiUo8ehPeTlhvttT0


## list top ranking videos

In [77]:
request = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        chart="mostPopular",
        regionCode="US",
        maxResults = 1000
    )
response = request.execute()

In [87]:
response['items'][0]['statistics']

{'commentCount': '8192',
 'dislikeCount': '1968',
 'favoriteCount': '0',
 'likeCount': '131285',
 'viewCount': '1938201'}

In [95]:
# encapsulate
def get_top_ranking_videos():
  prev = []
  try:
    with open(TOP_VIDEOS) as fp:
      prev = json.loads(fp.read())
  except:
    pass
  if len(prev) > 0 and (datetime.now() - datetime.strptime(prev[-1], "%d,%m,%Y,%H,%M")).hour < 1:
    return prev[:-1]
  else:
    total_response = []
    request = youtube.videos().list(
          part="snippet,contentDetails,statistics",
          chart="mostPopular",
          regionCode="US"
      )
    response = request.execute()

    pageInfo = response['pageInfo']
    total_response.append(response['items'])
    while pageInfo['totalResults'] > 0:
      for i in response['items']:
        insert_to_videos(i)
      nextToken = response.get('nextPageToken', 'UNKNOWN')
      pageInfo['totalResults'] -= pageInfo['resultsPerPage']
      if pageInfo['totalResults'] > 0:
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            chart="mostPopular",
            regionCode="US",
            pageToken = nextToken
        )
        response = request.execute()
    total_response.append(datetime.now().strftime("%d,%m,%Y,%H,%M"))
    with open(TOP_VIDEOS) as fp:
      json.dump(total_response, fp)


In [113]:
now = datetime.now()
now

datetime.datetime(2020, 12, 5, 2, 55, 29, 951945)

In [114]:
datetime.strptime(now.strftime("%d,%m,%Y,%H,%M"), "%d,%m,%Y,%H,%M")

datetime.datetime(2020, 12, 5, 2, 55)

In [96]:
get_top_ranking_videos()

In [98]:
# test our return
conn = sqlite3.connect(YOUTUBE_DBNAME)
cur = conn.cursor()
for video in cur.execute('select * from videos').fetchall():
  print(video)
conn.close()

('ieTnO1Dvt-I', 'Lil Baby - On Me (Official Video)', 10, '1963510', '131797', '1994', '0', '8212')
('G2bx3FzgJ6o', 'Mariah Carey - Oh Santa! (Official Music Video) ft. Ariana Grande, Jennifer Hudson', 10, '4963966', '527649', '8744', '0', '41603')
('pKtUpBjIl-g', 'Lil Baby - Errbody (Official Video)', 10, '1881609', '128299', '1731', '0', '10221')
('vMLk_T0PPbk', 'Black Eyed Peas, Shakira - GIRL LIKE ME (Official Music Video)', 10, '2205086', '235846', '7308', '0', '19769')
('N_kp7qqUk5Y', 'Clash Royale: NEW LEGENDARY CARD REVEAL 😮 TV Royale Update', 20, '2689151', '133974', '3080', '0', '6794')
('VZzZKuQUguk', "I'm Done Making Good Videos", 20, '2266654', '402795', '3618', '0', '26572')
('MTgKfVXpMBY', "OUR GENDER REVEAL! | The Herbert's", 22, '2938557', '111394', '2806', '0', '7938')
('RfcY3J6CjgQ', 'Swatching EVERY Liquid Lipstick I’ve EVER Made', 26, '388823', '32870', '1385', '0', '5285')
('O__Lo2FQJKo', 'No More CLASHMAS?!', 20, '15150883', '225127', '8043', '0', '8170')
('5f6mDf

## search channel by name

In [99]:
request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        forUsername="VaatiVidya"
    )

response = request.execute()

In [100]:
response

{'etag': 'Ea3cAE4KHCCN8DhzhTWeYE7e_Mg',
 'items': [{'contentDetails': {'relatedPlaylists': {'favorites': '',
     'likes': '',
     'uploads': 'UUe0DNp0mKMqrYVaTundyr9w'}},
   'etag': 'Lw1Jb88WjbNSZ5dd9WYY4Xf1P8I',
   'id': 'UCe0DNp0mKMqrYVaTundyr9w',
   'kind': 'youtube#channel',
   'snippet': {'country': 'AU',
    'customUrl': 'vaatividya',
    'description': "Subscribe for regular DARK SOULS, DARK SOULS 2, DARK SOULS 3, DEMON'S SOULS, BLOODBORNE, and SEKIRO: SHADOWS DIE TWICE content!..Lore..Gameplay..Music..Animation..\n\nEvery video I upload is held to a certain standard. I emphasise Quality over Quantity and I don't want to clog up your feed with mediocre videos.\n\n====================\n\n-Aim for one video per week\n\n-Souls-like titles are at the heart of this channel\n\n-This channel is supported primarily through Patreon.com/vaatividya, not advertisements!\n\n====================\n\nHave a question or comment related to my videos? I respond best on ask-vaati, a Discord text 

In [101]:
# insert the channel
def search_channel_by_name(channel):
  request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        forUsername="VaatiVidya"
    )

  response = request.execute()
  if response['pageInfo']['totalResults'] > 0:
    response = response['items'][0]
    insert_to_channels(response)
    return response

## search all playlists in the channel

In [33]:
request = youtube.playlists().list(
        part="snippet,contentDetails",
        channelId="UCe0DNp0mKMqrYVaTundyr9w",
        maxResults=25
    )
response = request.execute()

In [34]:
response['items'][8]

{'contentDetails': {'itemCount': 6},
 'etag': '_rboJCbBqqCVb8rjzWlsYBAh5ZQ',
 'id': 'PLWLedd0Zw3c6U1WRZBx6eZ-BXprlfQelv',
 'kind': 'youtube#playlist',
 'snippet': {'channelId': 'UCe0DNp0mKMqrYVaTundyr9w',
  'channelTitle': 'VaatiVidya',
  'description': '',
  'localized': {'description': '', 'title': 'Sekiro Lore'},
  'publishedAt': '2019-05-03T04:15:44Z',
  'thumbnails': {'default': {'height': 90,
    'url': 'https://i.ytimg.com/vi/9OmfWCvwM9g/default.jpg',
    'width': 120},
   'high': {'height': 360,
    'url': 'https://i.ytimg.com/vi/9OmfWCvwM9g/hqdefault.jpg',
    'width': 480},
   'maxres': {'height': 720,
    'url': 'https://i.ytimg.com/vi/9OmfWCvwM9g/maxresdefault.jpg',
    'width': 1280},
   'medium': {'height': 180,
    'url': 'https://i.ytimg.com/vi/9OmfWCvwM9g/mqdefault.jpg',
    'width': 320},
   'standard': {'height': 480,
    'url': 'https://i.ytimg.com/vi/9OmfWCvwM9g/sddefault.jpg',
    'width': 640}},
  'title': 'Sekiro Lore'}}

## search all videos in a playlist

This playlist has 6 items, and is displayed on 2 pages.

In [36]:
request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId="PLWLedd0Zw3c6U1WRZBx6eZ-BXprlfQelv"
    )
response = request.execute()

In [37]:
response

{'etag': 'y_VazzI_vwCRn1Eeky5-MNTxroA',
 'items': [{'contentDetails': {'videoId': '9OmfWCvwM9g',
    'videoPublishedAt': '2019-05-03T04:17:03Z'},
   'etag': 'MF-8j5cV1wgXAskgMTgBL4DDTDM',
   'id': 'UExXTGVkZDBadzNjNlUxV1JaQng2ZVotQlhwcmxmUWVsdi41NkI0NEY2RDEwNTU3Q0M2',
   'kind': 'youtube#playlistItem',
   'snippet': {'channelId': 'UCe0DNp0mKMqrYVaTundyr9w',
    'channelTitle': 'VaatiVidya',
    'description': "It's lore time\nSpecial thanks to:\nTheParryGod, for the incredible footage in this vid ►https://bit.ly/2GWx3QS\n►ZullietheWitch, for her modding work - https://bit.ly/2O2p8Ic\n►Sanadsk, for extracting the dialogue - https://bit.ly/2bgwFLD\n►Thumbnail art by John Devlin - https://bit.ly/2J02QEa\n\n\nMERCHANDISE [buy a shirt]\n►Check out the new Teespring design: https://bit.ly/2LsWkH6\n\n\nKINDLE THE CHANNEL [become a patron]\n►http://www.patreon.com/vaatividya\n\n\nSUBSCRIBE TO HUMBLE [cheap monthly games]\n►https://www.humblebundle.com/monthly?partner=vaatividya\n\n\nTRANSLATE 

In [52]:
request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId="PLWLedd0Zw3c6U1WRZBx6eZ-BXprlfQelv",
        pageToken = "CAUQAA"
    )
response = request.execute()


In [53]:
response

{'etag': 'iog4tKJSDiCVy-LNk-SRZv_xO1M',
 'items': [{'contentDetails': {'videoId': 'x8IfGeHAq4o',
    'videoPublishedAt': '2019-08-20T01:35:05Z'},
   'etag': 'gr6hPIpub-1dECR1XR43yMNLhF0',
   'id': 'UExXTGVkZDBadzNjNlUxV1JaQng2ZVotQlhwcmxmUWVsdi4xMkVGQjNCMUM1N0RFNEUx',
   'kind': 'youtube#playlistItem',
   'snippet': {'channelId': 'UCe0DNp0mKMqrYVaTundyr9w',
    'channelTitle': 'VaatiVidya',
    'description': "Folklore best lore\n►Read more from Zirael - https://www.reddit.com/user/ziraelxx\n►Merchandise by Vaati at https://vaatividya.com/\n►Find Zirael on VK - https://vk.com/ziraelxx\n\nKINDLE THE CHANNEL [become a patron]\n►http://www.patreon.com/vaatividya\n\n\nSUBSCRIBE TO HUMBLE [cheap monthly games]\n►https://www.humblebundle.com/monthly?partner=vaatividya\n\n\nTRANSLATE MY VIDEOS\n►Know a second language? You could help bring my videos to a wider audience: http://bit.ly/2fB71WW\n\n\nPLAYLISTS\n►My Best Videos: http://bit.ly/2oMp66a\n►Elden Ring News: https://bit.ly/2ZkDV2d\n►Sek

In [54]:
request = youtube.videos().list(
    part = "snippet,contentDetails,statistics",
    id = response['items'][0]['contentDetails']['videoId']
)
response = request.execute()

In [55]:
response

{'etag': 'DQWdRbPbmADvTms45mz3hHS4Tnw',
 'items': [{'contentDetails': {'caption': 'false',
    'contentRating': {},
    'definition': 'hd',
    'dimension': '2d',
    'duration': 'PT14M25S',
    'licensedContent': True,
    'projection': 'rectangular'},
   'etag': 'ZVnYeGnYklcIoJ3njAQz3c-Twok',
   'id': 'x8IfGeHAq4o',
   'kind': 'youtube#video',
   'snippet': {'categoryId': '20',
    'channelId': 'UCe0DNp0mKMqrYVaTundyr9w',
    'channelTitle': 'VaatiVidya',
    'defaultAudioLanguage': 'en',
    'defaultLanguage': 'en',
    'description': "Folklore best lore\n►Read more from Zirael - https://www.reddit.com/user/ziraelxx\n►Merchandise by Vaati at https://vaatividya.com/\n►Find Zirael on VK - https://vk.com/ziraelxx\n\nKINDLE THE CHANNEL [become a patron]\n►http://www.patreon.com/vaatividya\n\n\nSUBSCRIBE TO HUMBLE [cheap monthly games]\n►https://www.humblebundle.com/monthly?partner=vaatividya\n\n\nTRANSLATE MY VIDEOS\n►Know a second language? You could help bring my videos to a wider aud

## search by keyword

Too many results sometimes. Only get the top 1000 results.

In [26]:
request = youtube.search().list(
        part="snippet",
        maxResults=25,
        q="dark soul"
    )
response = request.execute()

In [27]:
response['pageInfo']

{'resultsPerPage': 25, 'totalResults': 1000000}

In [28]:
response['nextPageToken']

'CBkQAA'

In [29]:
request = youtube.search().list(
        part="snippet",
        maxResults=25,
        q="dark soul",
        pageToken='CBkQAA'
    )
response = request.execute()
response

{'etag': '1X41kUfYK6jtcIfKeO6aORg9DdA',
 'items': [{'etag': 'Adt6mKZapYsTQd3p9f2fjSoCUYs',
   'id': {'kind': 'youtube#video', 'videoId': 'gg9X38z1a2U'},
   'kind': 'youtube#searchResult',
   'snippet': {'channelId': 'UC4oo36VSTk5fH6YJSICEDMA',
    'channelTitle': 'Marz',
    'description': "In part 1, we begin our journey to fulfill a prophecy as a Hollow in the Undead Asylum. Once chosen, we head to the ancient land of Lordran and it's Firelink ...",
    'liveBroadcastContent': 'none',
    'publishTime': '2020-07-12T15:07:20Z',
    'publishedAt': '2020-07-12T15:07:20Z',
    'thumbnails': {'default': {'height': 90,
      'url': 'https://i.ytimg.com/vi/gg9X38z1a2U/default.jpg',
      'width': 120},
     'high': {'height': 360,
      'url': 'https://i.ytimg.com/vi/gg9X38z1a2U/hqdefault.jpg',
      'width': 480},
     'medium': {'height': 180,
      'url': 'https://i.ytimg.com/vi/gg9X38z1a2U/mqdefault.jpg',
      'width': 320}},
    'title': 'The Land of Lordran | Dark Souls Remastered Pt

## save search results

Create 3 tables: **channels** for storing channel information, **playlists** for storing playlist information (foreign key channelId), **playlistItems** for sotring video information (foreign key playlistId).

These 3 tables should be created when we call the related APIs(shown above)

In [30]:
## create the database

# Bilibili API

In [ ]:
# bilibili part
import requests

In [ ]:
# basic information of an uploader
params = {
    "mid":25877935,
    "jsonp":"jsonp"
}
base_url = "https://api.bilibili.com/x/space/upstat"
response = requests.get(base_url, params=params)

In [ ]:
response.text

'{"code":0,"message":"0","ttl":1,"data":{}}'

In [ ]:
# basic information of a video
base_url = "http://api.bilibili.com/archive_stat"
params = {
    "bvid":"BV1As411U7Uz",
    "type":"jsonp"
}
response = requests.get(base_url, params=params)
response.text

'Fatal: API not support'